for 50episodes ,test number of collisions in dqn vs claude

TODO:
add metric for stopping ,lagging behind as well

In [1]:
import gymnasium as gym
import highway_env

# Agent
from stable_baselines3 import DQN


import sys
from tqdm.notebook import trange

from stable_baselines3 import DQN
import pprint
from matplotlib import pyplot as plt
import numpy as np
import joblib
from tqdm import trange
import os

2025-04-29 22:51:41.718196: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-29 22:51:41.720269: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-04-29 22:51:41.763518: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-04-29 22:51:42.936615: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
import base64
from pathlib import Path

from gymnasium.wrappers import RecordVideo
from IPython import display as ipythondisplay
from pyvirtualdisplay import Display



def record_videos(env, video_folder="videos"):
    wrapped = RecordVideo(
        env, video_folder=video_folder, episode_trigger=lambda e: True
    )

    # Capture intermediate frames
    env.unwrapped.set_record_video_wrapper(wrapped)

    return wrapped


def show_videos(path="videos"):
    html = []
    for mp4 in Path(path).glob("*.mp4"):
        video_b64 = base64.b64encode(mp4.read_bytes())
        html.append(
            """<video alt="{}" autoplay
                      loop controls style="height: 400px;">
                      <source src="data:video/mp4;base64,{}" type="video/mp4" />
                 </video>""".format(
                mp4, video_b64.decode("ascii")
            )
        )
    ipythondisplay.display(ipythondisplay.HTML(data="<br>".join(html)))


In [ ]:
##DQN MODEL
modelDqn = DQN.load("models/dqn_model.zip")
##
modelDqnllm = DQN.load("models/prachit_model_claude_highway.zip")


In [6]:
from gymnasium.wrappers import RecordVideo

config = {
            "observation": {
                "type": "Kinematics",
                "features": ["presence", "x", "y", "vx", "vy"],
                "absolute": True,
                "normalize": False,
                "see_behind": True,
                "duration": 11,
                # "vehicles_count": 4,
                "features_range": {
                        "x": [-100, 100],
                        "y": [-100, 100],
                        "vx": [-15, 15],
                        "vy": [-15, 15],
                    },
            },
            "action": {
                "type": "DiscreteMetaAction",
                "target_speeds": np.linspace(0, 32, 9),
            },
            "duration": 40,
            "vehicles_density": 1.25,
            "show_trajectories": True,
            "render_agent": True,
        }

# Load the trained DQN model
modelDqn = DQN.load("models_try/dqn_model_roundabout")

# Create environment
env = gym.make('roundabout-v0', render_mode='rgb_array', config=config)

total_score = 0
num_collisions = 0

for episode in trange(100, desc='Evaluate Episodes'):
    (obs, info), done, truncated = env.reset(), False, False
    episode_score = 0  # Track the episode's score
    collision_occurred = False
    
    while not (done or truncated):
        action, _ = modelDqn.predict(obs, deterministic=True)
        obs, reward, done, truncated, info = env.step(int(action))
        episode_score += reward  # Accumulate reward for the episode
        
        # Check for collision
        if "crashed" in info and info["crashed"]:
            collision_occurred = True
    
    if collision_occurred:
        num_collisions += 1
    
    total_score += episode_score  # Add to total score

env.close()
print(f'Total Score: {total_score}, Average Score per Episode: {total_score / 50:.2f}')
print(f'Total Collisions: {num_collisions}, Average Collisions per Episode: {num_collisions / 50:.2f}')



Evaluate Episodes: 100%|██████████| 100/100 [04:17<00:00,  2.57s/it]

Total Score: 2313.7499999999995, Average Score per Episode: 46.27
Total Collisions: 36, Average Collisions per Episode: 0.72


In [5]:
def extract_features_from_dataset(row,prev_action):
    """
    Extract features from the dataset based on the given criteria.
    """
    processed_data = []

    # Ego vehicle features
    ego_features = row[:5]
    ego_lane =row[29] # Lane ID of ego vehicle
    ego_speed = row[4]  # Speed of ego vehicle

    # Other vehicles' features
    other_vehicles = row[5:25].reshape(4, 5)  # 3 vehicles, 5 features each
    
    # Separate features of other vehicles
    other_lanes = row[25:29]  # Lane IDs of other vehicles
    distances = np.abs(other_vehicles[:, 2] - ego_features[2])  # Distances from ego vehicle
    relative_velocities = other_vehicles[:, 4] - ego_speed  # Relative velocities

    # Number of vehicles in ego lane and adjacent lanes
    vehicles_in_ego_lane = np.sum(other_lanes == ego_lane)
    vehicles_in_left_lane = np.sum(other_lanes == ego_lane - 1)
    vehicles_in_right_lane = np.sum(other_lanes == ego_lane + 1)

    ## Closest vehicles
    closest_ego_index = np.where(other_lanes == ego_lane, distances, np.inf).argmin() if vehicles_in_ego_lane != 0 else np.nan
    closest_left_index = np.where(other_lanes == ego_lane - 1, distances, np.inf).argmin() if vehicles_in_left_lane != 0 else np.nan
    closest_right_index = np.where(other_lanes == ego_lane + 1, distances, np.inf).argmin() if vehicles_in_right_lane != 0 else np.nan

    # Distances of other vehicles
    ## Ego lane
    if np.isnan(closest_ego_index):
        closest_in_ego_lane_dist = 10000  # Assign large value for no vehicle
        relative_velocity_ego_lane = 10000
    else:
        closest_in_ego_lane_dist = distances[closest_ego_index]
        relative_velocity_ego_lane = relative_velocities[closest_ego_index]
    
    ## Left lane
    if np.isnan(closest_left_index):
        # Check if the left lane is non-existent (i.e., topmost lane)
        if ego_lane == 1:
            closest_left_lane_dist = 0  # No lane to the left of topmost lane
            relative_velocity_left_lane = 0  
        else:
            closest_left_lane_dist = 10000  # No vehicle in left lane
            relative_velocity_left_lane = 10000  # No vehicle in left lane
    else:
        closest_left_lane_dist = distances[closest_left_index]
        relative_velocity_left_lane = relative_velocities[closest_left_index]
    
    ## Right lane
    if np.isnan(closest_right_index):
        # Check if the right lane is non-existent (i.e., bottommost lane)
        if ego_lane == 4:
            closest_right_lane_dist = 0  # No lane to the right of bottommost lane
            relative_velocity_right_lane = 0  # No vehicle in non-existent lane
        else:
            closest_right_lane_dist = 10000  # No vehicle in right lane
            relative_velocity_right_lane = 10000  # No vehicle in right lane
    else:
        closest_right_lane_dist = distances[closest_right_index]
        relative_velocity_right_lane = relative_velocities[closest_right_index]

    previous_action_1 = prev_action
    # Append computed features
    processed_data.append([
        vehicles_in_ego_lane,
        vehicles_in_left_lane,
        vehicles_in_right_lane,
        closest_in_ego_lane_dist,
        closest_left_lane_dist,
        closest_right_lane_dist,
        relative_velocity_ego_lane,
        relative_velocity_left_lane,
        relative_velocity_right_lane,
        previous_action_1,
        #previous_action_2,
        #actions
        ])


    return np.array(processed_data)

In [ ]:
def process(env, obs,prev_action):
    ##load model
  
    ego_vehicle = env.env.unwrapped.vehicle
    ego_position = ego_vehicle.position
    ego_heading = ego_vehicle.heading
    ego_lane = env.env.unwrapped.road.network.get_closest_lane_index(ego_position, ego_heading)[2]

    # Get all other vehicles
    all_vehicles = env.env.unwrapped.road.vehicles
    other_vehicles = [v for v in all_vehicles if v is not ego_vehicle]

    # Get the lane index of each other vehicle
    veh_lanes = []
    for v in other_vehicles:
        v_lane =env.env.unwrapped.road.network.get_closest_lane_index(v.position, v.heading)[2]
        veh_lanes.append(v_lane)

    # Count how many vehicles share the ego lane
    veh_lanes = np.array(veh_lanes)

    # -------------------------------------------
    # Append these features to the observation
    obs_flat = obs.flatten()
    obs_augmented =  np.concatenate([obs_flat, [ego_lane], veh_lanes])
  
    obs_processed = extract_features_from_dataset(obs_augmented,prev_action)
    #processed_obs.append(obs_processed)
    return obs_processed


In [ ]:
def rf_query(env, obs, prev_action):
    # Load the models
    rf_model_binary = joblib.load('models_try/binary_rf_model_collision_free_upsampled.pkl')
    rf_model_major = joblib.load("models_try/major_rf_model_tuned.pkl")
      # Flatten and reshape observation
    obs_processed = process(env, obs, prev_action)
    
    # Get the binary prediction and probabilities
    binary_pred = rf_model_binary.predict(obs_processed)[0]
    binary_pred_prob = rf_model_binary.predict_proba(obs_processed)[0]
    
    # Get the minor or major class prediction and probabilities
    if binary_pred == 0:
        minor_pred = 2
        minor_pred_prob = 1
        return [binary_pred, binary_pred_prob, minor_pred, minor_pred_prob]
    else:
        major_pred = rf_model_major.predict(obs_processed)[0]
        major_pred_prob = rf_model_major.predict_proba(obs_processed)[0]
        return [binary_pred, binary_pred_prob, major_pred, major_pred_prob]

In [7]:
import os
import gymnasium as gym
import numpy as np
from gymnasium.wrappers import RecordVideo
from tqdm import trange  # For progress tracking
from IPython.display import display, Video
import highway_env

# Base setting

# Environment configuration
config = {
            "observation": {
                "type": "Kinematics",
                "features": ["presence", "x", "y", "vx", "vy"],
                "absolute": True,
                "normalize": False,
                "see_behind": True,
                "duration": 11,
                # "vehicles_count": 4,
                "features_range": {
                        "x": [-100, 100],
                        "y": [-100, 100],
                        "vx": [-15, 15],
                        "vy": [-15, 15],
                    },
            },
            "action": {
                "type": "DiscreteMetaAction",
                "target_speeds": np.linspace(0, 32, 9),
            },
            "duration": 40,
            "vehicles_density": 1.25,
            "show_trajectories": True,
            "render_agent": True,
        }
        #


# Create directory for action predictions storage if it doesn't exist
# Create directory for action predictions storage if it doesn't exist
predictions_dir = 'predictions/roundabout_analysis'
if not os.path.exists(predictions_dir):
    os.makedirs(predictions_dir)

# Create and configure environment
env = gym.make("roundabout-v0",render_mode='rgb_array', config= config)
# Wrap environment with video recording
env = RecordVideo(env, video_folder=video_dir, episode_trigger=lambda episode_id: True)

prev_action = 4
num_collisions = 0
total_score = 0
num_episodes = 50

# Run test episodes and record them
for episode in trange(num_episodes, desc='Evaluating episodes'):
    collision_occurred = False
    episode_score = 0
    
    actions_file_path = os.path.join(predictions_dir, f'episode_{episode}_actions_normal.txt')
    
    with open(actions_file_path, 'w') as action_file:
        (obs, info), done, truncated = env.reset(), False, False
        while not (done or truncated):
            Class, Class_prob, action, action_prob = rf_query(obs, prev_action)
            obs, reward, done, truncated, info = env.step(int(action))
            episode_score += reward  # Accumulate episode score
            
            # Log the action and probabilities
            action_file.write(f"Class = {Class} (Probability: {Class_prob}) Action: {action} (Probability: {action_prob})\n")
            
            prev_action = action
            
            # Check for collision
            if "crashed" in info and info["crashed"]:
                collision_occurred = True
                print(f"Collision occurred in episode {episode}")

    if collision_occurred:
        num_collisions += 1
    total_score += episode_score

# Close the environment
env.close()

# Print results
print(f"Total Collisions: {num_collisions}/{num_episodes}")
print(f"Total Score: {total_score}")
print(f"Average Score per Episode: {total_score / num_episodes:.2f}")
print(f"Collision Rate: {num_collisions / num_episodes:.2%}")


Evaluating episodes:   0%|          | 0/50 [00:00<?, ?it/s]

/home/prachit/.local/lib/python3.8/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/home/prachit/.local/lib/python3.8/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/home/prachit/.local/lib/python3.8/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/home/prachit/.local/lib/python3.8/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/home/prachit/.local/lib/python3.8/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/home/prachit/.local/lib/pytho

Collision occurred in episode 0


/home/prachit/.local/lib/python3.8/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/home/prachit/.local/lib/python3.8/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/home/prachit/.local/lib/python3.8/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/home/prachit/.local/lib/python3.8/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/home/prachit/.local/lib/python3.8/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/home/prachit/.local/lib/pytho

Collision occurred in episode 8


...

Collision occurred in episode 41


/home/prachit/.local/lib/python3.8/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/home/prachit/.local/lib/python3.8/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/home/prachit/.local/lib/python3.8/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/home/prachit/.local/lib/python3.8/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/home/prachit/.local/lib/python3.8/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/home/prachit/.local/lib/pytho

Collision occurred in episode 42


/home/prachit/.local/lib/python3.8/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/home/prachit/.local/lib/python3.8/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/home/prachit/.local/lib/python3.8/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/home/prachit/.local/lib/python3.8/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/home/prachit/.local/lib/python3.8/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/home/prachit/.local/lib/pytho

Total Collisions: 16/50
Total Score: 1120.7955032888863
Average Score per Episode: 22.42
Collision Rate: 32.00%


In [8]:
from gymnasium.wrappers import RecordVideo

# Base setting
vehicleCount = 10

# Environment setting
config = {
    "observation": {
        "type": "Kinematics",
        "features": ["presence", "x", "y", "vx", "vy"],
        "absolute": True,
        "normalize": True,
        "vehicles_count": vehicleCount,
        "see_behind": True,
    },
    "action": {
        "type": "DiscreteMetaAction",
        "target_speeds": np.linspace(0, 32, 9),
    },
    "duration": 40,
    "vehicles_density": 2.5,
    "show_trajectories": True,
    "render_agent": True,
}

# Load the trained DQN model
modelDqnllm = DQN.load("models/prachit_model_claude_highway")


video_dir = 'videos/dqn-claude_test1'
if not os.path.exists(video_dir):
    os.makedirs(video_dir)

# Create and configure environment
env = gym.make('highway-v0', render_mode='rgb_array', config=config)

# Wrap environment with video recording
env = RecordVideo(env, video_folder=video_dir, episode_trigger=lambda episode_id: True)

total_score = 0
num_collisions = 0

for episode in trange(3, desc='Evaluate Episodes'):
    (obs, info), done, truncated = env.reset(), False, False
    episode_score = 0  # Track the episode's score
    collision_occurred = False
    
    while not (done or truncated):
        action, _ = modelDqnllm.predict(obs, deterministic=True)
        obs, reward, done, truncated, info = env.step(int(action))
        episode_score += reward  # Accumulate reward for the episode
        
        # Check for collision
        if "crashed" in info and info["crashed"]:
            collision_occurred = True
    
    if collision_occurred:
        num_collisions += 1
    
    total_score += episode_score  # Add to total score

env.close()
print(f'Total Score: {total_score}, Average Score per Episode: {total_score / 50:.2f}')
print(f'Total Collisions: {num_collisions}, Average Collisions per Episode: {num_collisions / 50:.2f}')

show_videos()

/home/prachit/.local/lib/python3.8/site-packages/stable_baselines3/common/save_util.py:167: UserWarning: Could not deserialize object lr_schedule. Consider using `custom_objects` argument to replace this object.
Exception: code() takes at most 16 arguments (18 given)
  warnings.warn(
/home/prachit/.local/lib/python3.8/site-packages/stable_baselines3/common/save_util.py:167: UserWarning: Could not deserialize object exploration_schedule. Consider using `custom_objects` argument to replace this object.
Exception: code() takes at most 16 arguments (18 given)
  warnings.warn(
/home/prachit/.local/lib/python3.8/site-packages/gymnasium/wrappers/record_video.py:94: UserWarning: WARN: Overwriting existing videos at /home/prachit/Desktop/Reward-shaping-with-LLMS/videos/dqn-claude_test1 folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(
Evaluate Episodes:   0%|          | 0/3 [00:27<?, ?it/s]

Moviepy - Building video /home/prachit/Desktop/Reward-shaping-with-LLMS/videos/dqn-claude_test1/rl-video-episode-0.mp4.
Moviepy - Writing video /home/prachit/Desktop/Reward-shaping-with-LLMS/videos/dqn-claude_test1/rl-video-episode-0.mp4



Evaluate Episodes:  33%|███▎      | 1/3 [00:27<00:55, 27.97s/it]

Moviepy - Done !
Moviepy - video ready /home/prachit/Desktop/Reward-shaping-with-LLMS/videos/dqn-claude_test1/rl-video-episode-0.mp4


Evaluate Episodes:  33%|███▎      | 1/3 [00:28<00:55, 27.97s/it]

...

Moviepy - Done !
Moviepy - video ready /home/prachit/Desktop/Reward-shaping-with-LLMS/videos/dqn-claude_test1/rl-video-episode-1.mp4


Evaluate Episodes:  67%|██████▋   | 2/3 [00:57<00:12, 12.01s/it]

Moviepy - Building video /home/prachit/Desktop/Reward-shaping-with-LLMS/videos/dqn-claude_test1/rl-video-episode-2.mp4.
Moviepy - Writing video /home/prachit/Desktop/Reward-shaping-with-LLMS/videos/dqn-claude_test1/rl-video-episode-2.mp4



Evaluate Episodes: 100%|██████████| 3/3 [00:57<00:00, 19.24s/it]


Moviepy - Done !
Moviepy - video ready /home/prachit/Desktop/Reward-shaping-with-LLMS/videos/dqn-claude_test1/rl-video-episode-2.mp4
Total Score: 56.090014640344776, Average Score per Episode: 1.12
Total Collisions: 1, Average Collisions per Episode: 0.02
